In [ ]:
# @title Storm Regression Model Selection
import math
def mapping_series(data_np,window_size,window_hop):

  # Break series into a fixed window length
  data_np = data_np.values
  start_frame = window_size
  end_frame = window_hop * math.floor(float(len(data_np)) / window_hop)
  win_step = window_size - window_hop
  wd = []
  for frame_idx in range(start_frame, end_frame, win_step):
      window = data_np[frame_idx-window_size:frame_idx]
      wd.append(window)

  # Convert list to array
  w_props = np.array(wd)
  # Slice and convert array to temp dataframe
  temp_f = w_props[:,:,0]
  preci_f = w_props[:,:,1]
  gust_f = w_props[:,:,2]
  wind_f = w_props[:,:,3]
  windir_f = w_props[:,:,4]
  lai_f = w_props[:,:,5]

  # Build relationship with weather predictors
  # Take min and max of temperature
  t_min = DataFrame(temp_f)
  t_min = t_min.min(axis=1)
  t_max = DataFrame(temp_f)
  t_max = t_max.max(axis=1)
  # Take max and avg of precipitation
  p_max = DataFrame(preci_f)
  p_max = p_max.max(axis=1)
  p_avg = DataFrame(preci_f)
  p_avg = p_avg.mean(axis=1)
  # Take max and avg of gust
  g_max = DataFrame(gust_f)
  g_max = g_max.max(axis=1)
  g_avg = DataFrame(gust_f)
  g_avg = g_avg.mean(axis=1)
  # Take max and avg of wind
  w_max = DataFrame(wind_f)
  w_max = w_max.max(axis=1)
  w_avg = DataFrame(wind_f)
  w_avg = w_avg.mean(axis=1)
  # Take avg and std or variance (var) of wind direction
  wdir_std = DataFrame(windir_f)
  wdir_std = wdir_std.std(axis=1)
  wdir_avg = DataFrame(windir_f)
  wdir_avg = wdir_avg.mean(axis=1)
  # Take max or min of leaf area index
  lai_avg = DataFrame(lai_f)
  lai_avg = lai_avg.max(axis=1)

  # Convert weather predictors to numpy array
  t_min = np.array(t_min)
  t_max = np.array(t_max)
  p_max = np.array(p_max)
  p_avg = np.array(p_avg)
  g_max = np.array(g_max)
  g_avg = np.array(g_avg)
  w_max = np.array(w_max)
  w_avg = np.array(w_avg)
  wdir_avg = np.array(wdir_avg)
  wdir_std = np.array(wdir_std)
  lai_avg = np.array(lai_avg)

  # Concatenate weather predictors
  w_preds = np.concatenate([t_min[:, None],
                                    t_max[:, None],
                                    p_max[:, None],
                                    p_avg[:, None],
                                    g_max[:, None],
                                    g_avg[:, None],
                                    w_max[:, None],
                                    w_avg[:, None],
                                    wdir_avg[:, None],
                                    wdir_std[:, None],
                                    lai_avg[:, None],], axis=1)
  preds = pd.DataFrame(w_preds,columns=['Temp_min','Temp_max','Preci_max','Preci_avg',
                                        'Gust_max','Gust_avg','Wind_max','Wind_avg',
                                        'Windir_avg','Windir_std','LAI_avg'])
  return preds

#@title compute_samples_x_y_zero_train
def compute_samples_x_y_zero_train(vals_set,lag_ext,ff):
  # retrieve train data intervals of windows
  X_st_s = []
  y_st_s = []
  br = 1
  X_scaler = MinMaxScaler(feature_range=(0, 1))
  Y_scaler = MinMaxScaler(feature_range=(0, 1))

  if isinstance(ff[0], list):
    fff = ff[0]
    for c in range(fff[0],fff[1]+1):
      df_w_cv = vals_set[c]
      df_ws_cv = df_w_cv[['id','Temp','Preci','Gust','Wind','Windir','LAI']]
      df_outage_cv = df_w_cv[['id','Total Outages']]

      if df_ws_cv.isnull().values.any():
        # Interpolate missing values
        df_ws_cv = df_ws_cv.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
        df_ws_cv['id'] = df_ws_cv['id'].astype(int)
        df_outage_cv = df_outage_cv.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
        df_outage_cv['id'] = df_outage_cv['id'].astype(int)

      df_win_drop = df_ws_cv.drop('id', axis=1)
      df_outage_drop_cv = df_outage_cv.drop('id', axis=1)

      # add relative normalized time points
      pt_array = np.arange(len(df_win_drop))
      df_win_drop['norm_val'] = [(x+1)/len(df_win_drop) for x in pt_array]
      df_win_drop = df_win_drop[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]

      df_win_no_lags = df_win_drop.reset_index(drop=True)
      df_win_no_lags_vals = df_win_no_lags.values

      df_outage_drop_cv = df_outage_drop_cv.reset_index(drop=True)
      df_outage_drop_vals_cv = df_outage_drop_cv.values

      xtrain_storms_no_lags , ytrain_storms_no_lags = split_sequences_cv(df_win_no_lags_vals,
                                                                      df_outage_drop_vals_cv,lag_ext+1)
      X_st_s.append(xtrain_storms_no_lags)
      y_st_s.append(pd.DataFrame(ytrain_storms_no_lags))

    ffk = ff[1]
    for c in range(ffk[0],ffk[1]+1):
      df_w_cv = vals_set[c]
      df_ws_cv = df_w_cv[['id','Temp','Preci','Gust','Wind','Windir','LAI']]
      df_outage_cv = df_w_cv[['id','Total Outages']]

      if df_ws_cv.isnull().values.any():
        # Interpolate missing values
        df_ws_cv = df_ws_cv.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
        df_ws_cv['id'] = df_ws_cv['id'].astype(int)
        df_outage_cv = df_outage_cv.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
        df_outage_cv['id'] = df_outage_cv['id'].astype(int)

      df_win_drop = df_ws_cv.drop('id', axis=1)
      df_outage_drop_cv = df_outage_cv.drop('id', axis=1)

      # add relative normalized time points
      pt_array = np.arange(len(df_win_drop))
      df_win_drop['norm_val'] = [(x+1)/len(df_win_drop) for x in pt_array]
      df_win_drop = df_win_drop[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]


      df_win_no_lags = df_win_drop.reset_index(drop=True)
      df_win_no_lags_vals = df_win_no_lags.values

      df_outage_drop_cv = df_outage_drop_cv.reset_index(drop=True)
      df_outage_drop_cv_vals = df_outage_drop_cv.values

      xtrain_storms_no_lags , ytrain_storms_no_lags = split_sequences_cv(df_win_no_lags_vals,
                                                                      df_outage_drop_cv_vals,lag_ext+1)
      X_st_s.append(xtrain_storms_no_lags)
      y_st_s.append(pd.DataFrame(ytrain_storms_no_lags))

    sample_X = np.vstack(X_st_s)
    sample_y = pd.concat(y_st_s)
    sample_X = np.array(sample_X)
    xtr_reshape = sample_X.reshape(sample_X.shape[0],sample_X.shape[1]*sample_X.shape[-1])

    # Normalize regression varaibles
    xtr_scale =   X_scaler.fit_transform(xtr_reshape)
    sample_y_scale = Y_scaler.fit_transform(sample_y)
    return (xtr_scale,sample_y_scale,X_scaler,Y_scaler)

  else:
    X_st_s = []
    y_st_s = []
    br = 1
    X_scaler = MinMaxScaler(feature_range=(0, 1))
    Y_scaler = MinMaxScaler(feature_range=(0, 1))

    for c in range(ff[0],ff[1]+1):
      df_w_cv = vals_set[c]
      df_ws_cv = df_w_cv[['id','Temp','Preci','Gust','Wind','Windir','LAI']]
      df_outage_cv = df_w_cv[['id','Total Outages']]

      if df_ws_cv.isnull().values.any():
        # Interpolate missing values
        df_ws_cv = df_ws_cv.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
        df_ws_cv['id'] = df_ws_cv['id'].astype(int)
        df_outage_cv = df_outage_cv.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
        df_outage_cv['id'] = df_outage_cv['id'].astype(int)

      df_win_drop = df_ws_cv.drop('id', axis=1)
      df_outage_drop_cv = df_outage_cv.drop('id', axis=1)

      # add relative normalized time points
      pt_array = np.arange(len(df_win_drop))
      df_win_drop['norm_val'] = [(x+1)/len(df_win_drop) for x in pt_array]
      df_win_drop = df_win_drop[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]

      df_win_no_lags = df_win_drop.reset_index(drop=True)
      df_win_no_lags_vals = df_win_no_lags.values

      df_outage_drop_cv = df_outage_drop_cv.reset_index(drop=True)
      df_outage_drop_cv_vals = df_outage_drop_cv.values

      xtrain_storms_no_lags , ytrain_storms_no_lags = split_sequences_cv(df_win_no_lags_vals,
                                                                      df_outage_drop_cv_vals,lag_ext+1)
      X_st_s.append(xtrain_storms_no_lags)
      y_st_s.append(pd.DataFrame(ytrain_storms_no_lags))

    sample_X = np.vstack(X_st_s)
    sample_y = pd.concat(y_st_s)
    sample_X = np.array(sample_X)
    xtr_reshape = sample_X.reshape(sample_X.shape[0],sample_X.shape[1]*sample_X.shape[-1])

    # Normalize regression variables
    xtr_scale = X_scaler.fit_transform(xtr_reshape)
    sample_y_scale = Y_scaler.fit_transform(sample_y)
    return (xtr_scale,sample_y_scale,X_scaler,Y_scaler)

#@title ml val iforest
from sklearn.ensemble import IsolationForest
def ifor_vals(val_set):
    clf_val =IsolationForest(max_samples='auto',random_state=rng)
    clf_val.fit(val_set)
    wea_pd_val = pd.DataFrame(val_set)
    new_val_set = wea_pd_val.copy()
    new_val_set['scores'] = clf_val.decision_function(wea_pd_val)
    new_val_set['anomaly'] = clf_val.predict(wea_pd_val)
    new_val_set['anomaly'] = new_val_set['anomaly'].map({1:0,-1:1})
    return new_val_set

#@title compute_samples_x_y_zero_test
import numpy as np
def compute_samples_x_y_zero_test(vals_set,lag_ext,ff):
  # retrieve train data intervals of windows
  X_st_s = []
  y_st_s = []

  for c in range(ff[0],ff[1]+1):
    df_w_cv_test = vals_set[c]
    df_ws_cv_test = df_w_cv_test[['id','Temp','Preci','Gust','Wind','Windir','LAI']]
    df_outage_cv_test = df_w_cv_test[['id','Total Outages']]

    if df_ws_cv_test.isnull().values.any():
        # Interpolate missing values
        df_ws_cv_test = df_ws_cv_test.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
        df_ws_cv_test['id'] = df_ws_cv_test['id'].astype(int)

    df_win_drop = df_ws_cv_test.drop('id', axis=1)
    df_outage_drop_cv_test = df_outage_cv_test.drop('id', axis=1)

    # add relative normalized time points
    pte_array = np.arange(len(df_win_drop))
    df_win_drop['norm_val'] = [(x+1)/len(df_win_drop) for x in pte_array]
    df_win_drop = df_win_drop[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]

    df_win_no_lags = df_win_drop.reset_index(drop=True)
    df_win_no_lags_vals = df_win_no_lags.values

    df_outage_drop_cv_test = df_outage_drop_cv_test.reset_index(drop=True)
    df_outage_drop_cv_test_vals = df_outage_drop_cv_test.values

    xtrain_storms_no_lags , ytrain_storms_no_lags = split_sequences_cv(df_win_no_lags_vals,
                                                                    df_outage_drop_cv_test_vals,lag_ext+1)
    X_st_s.append(xtrain_storms_no_lags)
    y_st_s.append(pd.DataFrame(ytrain_storms_no_lags))

  sample_X = np.vstack(X_st_s)
  sample_y = pd.concat(y_st_s)
  sample_X = np.array(sample_X)
  xtr_reshape = sample_X.reshape(sample_X.shape[0],sample_X.shape[1]*sample_X.shape[-1])

  return (xtr_reshape,sample_y,y_st_s)


#@title compute_samples_x_y_train_multiple
import numpy as np
def compute_samples_x_y_train_multiple(vals_set,ff,lag_ext,raw_trr):
  X_st_s_full = []
  y_st_s_full = []

  X_scaler_full = MinMaxScaler(feature_range=(0, 1))
  Y_scaler_full = MinMaxScaler(feature_range=(0, 1))

  if isinstance(ff[0], list):
    fff = ff[0]
    for c in range(fff[0],fff[1]+1):
      df_w_cv = vals_set[c]
      df_ws_cv = df_w_cv[['id','Temp','Preci','Gust','Wind','Windir','LAI']]
      df_outage_cv = df_w_cv[['id','Total Outages']]
      df_outage_drop_cv = df_outage_cv.drop('id', axis=1)

      if df_ws_cv.isnull().values.any():
        # Interpolate missing values
        df_ws_cv = df_ws_cv.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
        df_ws_cv['id'] = df_ws_cv['id'].astype(int)

      df_lags = get_lags(df_ws_cv,lag_ext,raw_trr)
      df_win_drop = df_lags.drop('Total Outages', axis=1)

      # add relative normalized time points
      ptef_array = np.arange(len(df_win_drop))
      df_win_drop['norm_val'] = [(x+1)/len(df_win_drop) for x in ptef_array]
      df_win_drop = df_win_drop[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]

      df_win_no_lags = df_win_drop.reset_index(drop=True)
      df_win_no_lags_vals = df_win_no_lags.values

      df_outage_drop_cv = df_outage_drop_cv.reset_index(drop=True)
      df_outage_drop_cv_vals = df_outage_drop_cv.values

      xtrain_storms_no_lags , ytrain_storms_no_lags = split_sequences_cv(df_win_no_lags_vals,
                                                                      df_outage_drop_cv_vals,lag_ext+1)
      X_st_s_full.append(xtrain_storms_no_lags)
      y_st_s_full.append(pd.DataFrame(ytrain_storms_no_lags))

    ffk = ff[1]
    for d in range(ffk[0],ffk[1]+1):
      df_w_cv = vals_set[d]
      df_ws_cv = df_w_cv[['id','Temp','Preci','Gust','Wind','Windir','LAI']]
      df_outage_cv = df_w_cv[['id','Total Outages']]
      df_outage_drop_cv = df_outage_cv.drop('id', axis=1)

      if df_ws_cv.isnull().values.any():
        # Interpolate missing values
        df_ws_cv = df_ws_cv.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
        df_ws_cv['id'] = df_ws_cv['id'].astype(int)

      df_lags = get_lags(df_ws_cv,lag_ext,raw_trr)
      df_win_drop = df_lags.drop('Total Outages', axis=1)

      # add relative normalized time points
      ptef_array = np.arange(len(df_win_drop))
      df_win_drop['norm_val'] = [(x+1)/len(df_win_drop) for x in ptef_array]
      df_win_drop = df_win_drop[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]
      mask = df_win_drop

      df_win_no_lags = mask.reset_index(drop=True)
      df_win_no_lags_vals = df_win_no_lags.values

      df_outage_drop_cv = df_outage_drop_cv.reset_index(drop=True)
      df_outage_drop_cv_vals = df_outage_drop_cv.values

      xtrain_storms_no_lags , ytrain_storms_no_lags = split_sequences_cv(df_win_no_lags_vals,
                                                                      df_outage_drop_cv_vals,lag_ext+1)
      X_st_s_full.append(xtrain_storms_no_lags)
      y_st_s_full.append(pd.DataFrame(ytrain_storms_no_lags))

    sample_X = np.vstack(X_st_s_full)
    sample_y = pd.concat(y_st_s_full)
    sample_X = np.array(sample_X)
    xtr_reshape = sample_X.reshape(sample_X.shape[0],sample_X.shape[1]*sample_X.shape[-1])

    # Normalize regression variables
    xtr_scale = X_scaler_full.fit_transform(xtr_reshape)
    sample_y_scale = Y_scaler_full.fit_transform(sample_y)
    return (xtr_scale,sample_y_scale,X_scaler_full,Y_scaler_full)

  else:
    X_st_s_df = []
    y_st_s_df = []
    br = 1
    X_scaler_df = MinMaxScaler(feature_range=(0, 1))
    Y_scaler_df = MinMaxScaler(feature_range=(0, 1))

    for c in range(ff[0],ff[1]+1):
      df_w_cv = vals_set[c]
      df_ws_cv = df_w_cv[['id','Temp','Preci','Gust','Wind','Windir','LAI']]
      df_outage_cv = df_w_cv[['id','Total Outages']]
      df_outage_drop_cv = df_outage_cv.drop('id', axis=1)

      if df_ws_cv.isnull().values.any():
        # Interpolate missing values
        df_ws_cv = df_ws_cv.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
        df_ws_cv['id'] = df_ws_cv['id'].astype(int)

      df_lags = get_lags(df_ws_cv,lag_ext,raw_trr)
      df_win_drop = df_lags.drop('Total Outages', axis=1)

      # add relative normalized time points
      ptef_array = np.arange(len(df_win_drop))
      df_win_drop['norm_val'] = [(x+1)/len(df_win_drop) for x in ptef_array]
      df_win_drop = df_win_drop[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]

      df_win_no_lags = df_win_drop.reset_index(drop=True)
      df_win_no_lags_vals = df_win_no_lags.values

      df_outage_drop_cv = df_outage_drop_cv.reset_index(drop=True)
      df_outage_drop_cv_vals = df_outage_drop_cv.values

      xtrain_storms_no_lags , ytrain_storms_no_lags = split_sequences_cv(df_win_no_lags_vals,
                                                                      df_outage_drop_cv_vals,lag_ext+1)
      X_st_s_df.append(xtrain_storms_no_lags)
      y_st_s_df.append(pd.DataFrame(ytrain_storms_no_lags))

    sample_X = np.vstack(X_st_s_df)
    sample_y = pd.concat(y_st_s_df)
    sample_X = np.array(sample_X)
    xtr_reshape = sample_X.reshape(sample_X.shape[0],sample_X.shape[1]*sample_X.shape[-1])

    # Normalize regression variables
    xtr_scale = X_scaler_df.fit_transform(xtr_reshape)
    sample_y_scale = Y_scaler_df.fit_transform(sample_y)
    return (xtr_scale,sample_y_scale,X_scaler_df,Y_scaler_df)

#@title compute_samples_x_y_test_multiple
def compute_samples_x_y_test_multiple(vals_set,ff,lag_ext,raw_trr):
  # retrieve train data intervals of windows
  X_st_s_test = []
  y_st_s_test = []

  for c in range(ff[0],ff[1]+1):
    df_w_cv_test = vals_set[c]
    df_ws_cv_test = df_w_cv_test[['id','Temp','Preci','Gust','Wind','Windir','LAI']]
    df_outage_cv_test = df_w_cv_test[['id','Total Outages']]
    df_outage_drop_cv_test = df_outage_cv_test.drop('id', axis=1)

    if df_ws_cv_test.isnull().values.any():
      # Interpolate missing values
      df_ws_cv_test = df_ws_cv_test.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
      df_ws_cv_test['id'] = df_ws_cv_test['id'].astype(int)

    df_lags_test = get_lags(df_ws_cv_test,lag_ext,raw_trr)
    df_win_drop = df_lags_test.drop('Total Outages', axis=1)
    ##df_win_drop = df_ws_lags_test.drop('id', axis=1)

    # add relative normalized time points
    ptesf_array = np.arange(len(df_win_drop))
    df_win_drop['norm_val'] = [(x+1)/len(df_win_drop) for x in ptesf_array]
    df_win_drop = df_win_drop[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]

    df_win_no_lags = df_win_drop.reset_index(drop=True)
    df_win_no_lags_vals = df_win_no_lags.values

    df_outage_drop_cv_test = df_outage_drop_cv_test.reset_index(drop=True)
    df_outage_drop_cv_test_vals = df_outage_drop_cv_test.values

    xtrain_storms_no_lags , ytrain_storms_no_lags = split_sequences_cv(df_win_no_lags_vals,
                                                                    df_outage_drop_cv_test_vals,lag_ext+1)
    X_st_s_test.append(xtrain_storms_no_lags)
    y_st_s_test.append(pd.DataFrame(ytrain_storms_no_lags))

  sample_X = np.vstack(X_st_s_test)
  sample_y = pd.concat(y_st_s_test)
  sample_X = np.array(sample_X)
  xtr_reshape = sample_X.reshape(sample_X.shape[0],sample_X.shape[1]*sample_X.shape[-1])
  return (xtr_reshape,sample_y,y_st_s_test)

#@title k-folds
def k_folds(data,split_f):
  df_set = data[:]
  n_storms = len(df_set)
  splits = split_f
  n_folds = int(n_storms/splits)

  folds = []
  for i in range(splits):
      if i == 0:
        folds.append([i*(n_folds),(i+1)*(n_folds-1)])
      else:
        folds.append([i*(n_folds),((i+1)*(n_folds))-1])
  return folds

#@title Avg_CV
def Average_cv(lst):
    return sum(lst) / len(lst)

#@title outages - weather

import pandas as pd
from datetime import datetime
import numpy as np
from pandas import DataFrame
from scipy.stats import boxcox
pd.set_option('display.max_colwidth', None)


def outs_weas(g_data,g_high_out,r_val):
    # parameters
    min_ones = 1 # minimum amount of overlapped outliers (ones) between weather and customer outage outliers to call a match
    min_consecutive_outage_ones = 0 # minimum number of consecutive ones required to call a storm window

    raw_test_w = r_val.values
    rtest = DataFrame(raw_test_w)
    rtest.columns = ['id','Temp','Preci','Gust','Wind','Windir','LAI','Total Outages']
    rtest['id'] = rtest['id'].astype(int)
        rtest = rtest.drop('id', axis = 1).reset_index(drop=True)
    rtest['id'] = rtest.index
    rtest.columns = ['Temp','Preci','Gust','Wind','Windir','LAI','Total Outages','id']
    rtest = rtest[['id','Temp','Preci','Gust','Wind','Windir','LAI','Total Outages']]
    out_vals = rtest['Total Outages']
    wea_ifors_labs  = g_data['anomaly']
    out_probs_labs =  g_high_out['Storm_class']

    # parameters
    min_ones = 1 # minimum amount of overlapped outliers (ones) between weather and customer outage outliers to call a match
    min_consecutive_outage_ones = 2 # minimum number of consecutive ones required to call a storm window
    w = 5 # window size for calling outliers in the original data

    # compute consecutive ones
    def zero_runs(outlier_vector, threshold=2):
        # Create an array that is 1 where outlier_vector is 0, and pad each end with an extra 0.
        iszero = np.concatenate(([0], np.equal(outlier_vector, 1).view(np.int8), [0]))
        absdiff = np.abs(np.diff(iszero))
        # Runs start and end where absdiff is 1.
        ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
        filtered_ranges = []
        for r in ranges:
            if r[-1]-r[0]>=threshold:
                filtered_ranges.append(r)
        return np.asarray(filtered_ranges)

    # retrieve outlier values from storm intervals
    b_wea = wea_ifors_labs
    # retrieve and compute ranges of consecutive ones for outage intervals
    b_outage = out_probs_labs
    count_ones_outage = zero_runs(b_outage,min_consecutive_outage_ones)

    # customer outage and storm intervals matching algorithm
    # declare and initialize variables
    out_matches_s = [] # list that holds matched outage intervals indexes
    wea_matches_s = [] # list that holds matched weather outliers indexes
    visited = [0] # list that holds current indexes of matched weather outliers

    previously_used = np.zeros(len(out_probs_labs), dtype = np.int8) # previously visited position
    end_wea = len(b_wea)-1 # takes the length of the weather outliers staring from index 0
    end_out = len(count_ones_outage)-1 # takes the length of the outage intervals staring from index 0
    max_off_set = 24 # the max_hours of weather outliers before outage is reported

    """
    starting from the end of each data - outage intervals and storm outliers  -
    for each range of consecutive outage intervals, take max matches of at most 24hrs of past storm intervals
    """
    for ones in range(end_out,0,-1): # start from the end of the outage intervals, pick the last range of consective 1's
      # retrive indexes of the last outage range of consective 1's
      out_interval = count_ones_outage[ones]

      # while we haven't hit the off_set hrs on the weather outliers
      off_set = 0
      best_match = None # list that holds all offsets and matches in each iteration of outage intervals range
      max_val=0
      while off_set <= 23:
        # compute the number of matches in the two intervals
        num_matches=np.sum(b_wea[out_interval[0]-off_set:out_interval[1]-off_set])
        # check if this interval overlaps with a previous interval
        overlapped_with_previous=np.sum(previously_used[out_interval[0]-off_set:out_interval[1]-off_set])
        # store index position of matches, and matches in the matches list
        if(num_matches>max_val and overlapped_with_previous==0):
            best_match = (off_set, out_interval[0] - off_set, out_interval[1] - off_set, num_matches)
            max_val=num_matches
        # increment offset value or move to the next offset
        off_set = off_set + 1

      if best_match is not None and best_match[-1]>=min_ones:
          previously_used[best_match[1]:best_match[2]]=1
          # append indexes of max weather outliers matches to the weather matches list
          wea_matches_s.append((best_match[1], best_match[2]-1))
          # store the actual values indexes to the outage matches list
          out_matches_s.append((out_interval[0], out_interval[1] - 1))

    win_size = 5

    # Outage
    outage_s = []
    for outs in range(len(out_matches_s)):
        o_rtr = rtest.loc[(rtest['id']>=out_matches_s[outs][0]) & (rtest['id']<=out_matches_s[outs][-1]+win_size)]
        outage_s.append(o_rtr['Total Outages'])

    # weather
    weather_s = []
    for raws in range(len(wea_matches_s)):
        w_rtr = rtest.loc[(rtest['id']>=wea_matches_s[raws][0]) & (rtest['id']<=wea_matches_s[raws][-1]+win_size)]
        w_rtr.drop('Total Outages',axis=1,inplace=True)
        weather_s.append(w_rtr)

    # concatenate all data
    data_concats_s = []
    for i in range(len(weather_s)):
        if i >= len(weather_s)-1:
            break
        we = DataFrame(weather_s[i]).reset_index(drop=True)
        ou = DataFrame(outage_s[i]).reset_index(drop=True)
        datas = pd.concat([we,ou],axis=1)
        data_concats_s.append(datas)
    return  data_concats_s, rtest

#@title modeling training proportion
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
import patsy
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

def modeling_tr(all_dt_trp,las):

  X_st_s_vb_train = []
  y_st_s_vb_train = []

  train_len = len(all_dt_trp)
  vb_train = list([0,train_len-1])

  for vb_tr in range(vb_train[0],vb_train[1]+1):
    df_w = all_dt_trp[vb_tr]
    df_ws = df_w[['id','Temp','Preci','Gust','Wind','Windir','LAI']]
    df_outage = df_w[['id','Total Outages']]
    df_outage_drop = df_outage.drop('id', axis=1)

    if df_ws.isnull().values.any():
      # Interpolate missing values
      df_ws = df_ws.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
      df_ws['id'] = df_ws['id'].astype(int)

    ##df_lags = get_lags(df_ws,lag)
    ##df_win_drop_vb_train = df_lags.drop('Total Outages', axis=1)

    df_win_drop_vb_train = df_ws.drop('id', axis=1)

    # add relative normalized time points
    p_f_array = np.arange(len(df_win_drop_vb_train))
    df_win_drop_vb_train['norm_val'] = [(x+1)/len(df_win_drop_vb_train) for x in p_f_array]
    df_win_drop_vb_train = df_win_drop_vb_train[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]

    df_win_no_lags_vb_train = df_win_drop_vb_train.reset_index(drop=True)
    df_win_no_lags_vals_vb_train = df_win_no_lags_vb_train.values
    df_outage_drop = df_outage_drop.reset_index(drop=True)
    df_outage_drop_vals = df_outage_drop.values

    xtrain_storms_no_lags_vb_train , ytrain_storms_no_lags_vb_train = split_sequences_cv(df_win_no_lags_vals_vb_train,
                                                                                      df_outage_drop_vals,las+1)
    X_st_s_vb_train.append(xtrain_storms_no_lags_vb_train)
    y_st_s_vb_train.append(pd.DataFrame(ytrain_storms_no_lags_vb_train))

  sample_X_vb_train = np.vstack(X_st_s_vb_train)
  sample_y_b_train = pd.concat(y_st_s_vb_train)
  sample_X_vb_train = np.array(sample_X_vb_train)
  xtr_reshape_vb_train = sample_X_vb_train.reshape(sample_X_vb_train.shape[0],sample_X_vb_train.shape[1]*sample_X_vb_train.shape[-1])

  # Apply fit transforms on train data
  X_scaler_vb_train = MinMaxScaler(feature_range=(0, 1))
  xtr_scale_vb_train  = X_scaler_vb_train.fit_transform(xtr_reshape_vb_train)
  Y_scaler_vb_train = MinMaxScaler(feature_range=(0, 1))
  ytr_scale_vb_train = Y_scaler_vb_train.fit_transform(sample_y_b_train)

  d_x = DataFrame(xtr_scale_vb_train)
  d_y = DataFrame(ytr_scale_vb_train)
  data_fuse = pd.concat([d_x,d_y],axis=1)
  f = 'd_y~d_x - 1'
  y , X = patsy.dmatrices(f, data_fuse, return_type='matrix')
  ##rf_model = RandomForestRegressor(random_state=0).fit(X,y)
  rf_model = neighbors.KNeighborsRegressor().fit(X,y)
  return rf_model,X_scaler_vb_train,Y_scaler_vb_train

#@title modeling_lags training proportion
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
import patsy
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

def modeling_tr_lags(all_dt_trp,las,rwt):

  X_st_s_vb_train = []
  y_st_s_vb_train = []

  train_len = len(all_dt_trp)
  vb_train = list([0,train_len-1])

  for vb_tr in range(vb_train[0],vb_train[1]+1):
    df_w = all_dt_trp[vb_tr]
    df_ws = df_w[['id','Temp','Preci','Gust','Wind','Windir','LAI']]
    df_outage = df_w[['id','Total Outages']]
    df_outage_drop = df_outage.drop('id', axis=1)

    if df_ws.isnull().values.any():
      # Interpolate missing values
      df_ws = df_ws.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0)
      df_ws['id'] = df_ws['id'].astype(int)

    df_lags = get_lags_test(df_ws,las,rwt)
    df_win_drop_vb_train = df_lags.drop('Total Outages', axis=1)

    ##df_win_drop_vb_train = df_ws.drop('id', axis=1)

    # add relative normalized time points
    p_f_array = np.arange(len(df_win_drop_vb_train))
    df_win_drop_vb_train['norm_val'] = [(x+1)/len(df_win_drop_vb_train) for x in p_f_array]
    df_win_drop_vb_train = df_win_drop_vb_train[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]

    df_win_no_lags_vb_train = df_win_drop_vb_train.reset_index(drop=True)
    df_win_no_lags_vals_vb_train = df_win_no_lags_vb_train.values
    df_outage_drop = df_outage_drop.reset_index(drop=True)
    df_outage_drop_vals = df_outage_drop.values

    xtrain_storms_no_lags_vb_train , ytrain_storms_no_lags_vb_train = split_sequences_cv(df_win_no_lags_vals_vb_train,
                                                                                      df_outage_drop_vals,las+1)
    if not xtrain_storms_no_lags_vb_train:
      None
    else:
      X_st_s_vb_train.append(xtrain_storms_no_lags_vb_train)
      y_st_s_vb_train.append(pd.DataFrame(ytrain_storms_no_lags_vb_train))

  sample_X_vb_train = np.vstack(X_st_s_vb_train)
  sample_y_b_train = pd.concat(y_st_s_vb_train)
  sample_X_vb_train = np.array(sample_X_vb_train)
  xtr_reshape_vb_train = sample_X_vb_train.reshape(sample_X_vb_train.shape[0],sample_X_vb_train.shape[1]*sample_X_vb_train.shape[-1])

  # Apply fit transforms on train data
  X_scaler_vb_train = MinMaxScaler(feature_range=(0, 1))
  xtr_scale_vb_train  = X_scaler_vb_train.fit_transform(xtr_reshape_vb_train)
  Y_scaler_vb_train = MinMaxScaler(feature_range=(0, 1))
  ytr_scale_vb_train = Y_scaler_vb_train.fit_transform(sample_y_b_train)

  d_x = DataFrame(xtr_scale_vb_train)
  d_y = DataFrame(ytr_scale_vb_train)
  data_fuse = pd.concat([d_x,d_y],axis=1)
  f = 'd_y~d_x - 1'
  y , X = patsy.dmatrices(f, data_fuse, return_type='matrix')
  ##rf_model = RandomForestRegressor(random_state=0).fit(X,y)
  rf_model = neighbors.KNeighborsRegressor().fit(X,y)
  return rf_model,X_scaler_vb_train,Y_scaler_vb_train

#@title split sequences: cv
# split a multivariate sequence into samples
def split_sequences_cv(sequences, y_samples, n_steps):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    ##seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
    seq_x, seq_y = sequences[i:end_ix, :], y_samples[i]
    X.append(seq_x)
    y.append(seq_y)
  return X, y


#@title split a multivariate sequence: test data samples

def split_sequences_test(sequences,y_data,n_steps,off):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix + off > len(sequences):
    ##if end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], y_data[i + off]
    ##seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
    X.append(seq_x)
    y.append(seq_y)
  return X, y

#@title regression samples _ vals
import patsy
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf
from math import sqrt

#@title regression samples _ vals
import patsy
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf
from math import sqrt

def mod_vals(model,X_scaler_tr,Y_scaler_tr,data_c,lg):
  X_tests = []
  y_tests = []

  for full_tes in range(len(data_c)):
    df_ws_test_data = data_c[full_tes][['id','Temp','Preci','Gust','Wind','Windir','LAI']]
    df_outage_test_data_d = data_c[full_tes][['Total Outages']]

    df_last_drop = df_ws_test_data.drop('id', axis=1)

    if df_last_drop.empty:
      None
    else:
      # add relative normalized time points
      p_f_array = np.arange(len(df_last_drop))
      df_last_drop['norm_val'] = [(x+1)/len(df_last_drop) for x in p_f_array]
      df_last_drop = df_last_drop[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]

      df_last_drop = df_last_drop.reset_index(drop=True)
      df_last_drop_vals = df_last_drop.values

      df_outage_test_data_d = df_outage_test_data_d.reset_index(drop=True)
      df_outage_test_data_d_vals = df_outage_test_data_d.values

      xtest , ytest = split_sequences_cv(df_last_drop_vals,df_outage_test_data_d_vals,lg+1)

      if not xtest:
        None
      else:
        X_tests.append(xtest)
        y_tests.append(pd.DataFrame(ytest))

  sample_X_test = np.vstack(X_tests)
  sample_y_test = pd.concat(y_tests)
  sample_y_test = np.array(sample_y_test)
  sample_X_test_reshape = sample_X_test.reshape(sample_X_test.shape[0],sample_X_test.shape[1]*sample_X_test.shape[-1])

  #@title hourly outages: 2
  # Apply transforms on test data: hourly outages
  xtr_scale_test = X_scaler_tr.transform(sample_X_test_reshape)

  d_x_test = DataFrame(xtr_scale_test)
  d_y_test = DataFrame(sample_y_test).reset_index(drop=True)
  data_fuse_test = pd.concat([d_x_test,d_y_test],axis=1)
  f_test = 'd_y_test~d_x_test - 1'
  yst , Xst = patsy.dmatrices(f_test, data_fuse_test, return_type='matrix')
  X_test_apply = Xst
  y_test_apply = yst.astype(int)

  # make prediction on the transformed test data
  yhat = model.predict(X_test_apply)
  actual_counts = y_test_apply
  # Inverse response variable transform
  yhat_val = np.array(yhat)
  yhat_reshape = yhat_val.reshape(yhat_val.shape[0],1)
  predicted_counts = Y_scaler_tr.inverse_transform(yhat_reshape)

  # Convert to series
  actual_counts = np.array(actual_counts)
  ac = actual_counts.ravel()
  predicted_counts = np.array(predicted_counts)
  pc = predicted_counts.ravel()

  #OLS regression model with no intercept : just a straight line passing through the origin
  ac_df = DataFrame(ac)
  pc_df = DataFrame(pc)
  outputs = pd.concat([ac_df,pc_df], axis = 1)
  outputs.columns = ['true','fitted']

  # Form regression model specification with no intercept
  ols_expr = """true ~ fitted - 1"""
  # fit the OLS regression model
  aux_olsr_results = smf.ols(ols_expr, outputs).fit()
  # store R-squared in rmses list
  ##print('Test R-squared: %.3f' % (aux_olsr_results.rsquared))

  # Compute RMSE
  ac_df_rmse = DataFrame(ac)
  pc_df_rmse = DataFrame(pc)
  mse = mean_squared_error(ac_df_rmse, pc_df_rmse)
  rmse = sqrt(mse)

  #####################################
  # Total Outages
  #####################################
  wind_observ = []

  true_sum_totals = []
  for n in range(len(y_tests)):
    true_sum_totals.append(np.sum(y_tests[n]))
  tru_totals = pd.concat(true_sum_totals)
  tru_totals = tru_totals.reset_index(drop=True)

  pc_preds = DataFrame(pc)
  forecast = pc_preds
  preds_sum_totals = []
  win_index = 0
  for nk in range(len(y_tests)):
    win_len = len(y_tests[nk])
    # sum forecast windows
    preds_sum_totals.append(np.sum(forecast[win_index:win_index + win_len]))
    wind_observ.append(forecast[win_index:win_index + win_len])
    win_index = win_index + win_len
  preds_totals = pd.concat(preds_sum_totals)
  preds_totals = preds_totals.reset_index(drop=True)

  #OLS regression model with no intercept : just a straight line passing through the origin
  ac_totals = DataFrame(tru_totals)
  pc_totals = DataFrame(preds_totals)
  outputs_totals = pd.concat([ac_totals,pc_totals], axis = 1)
  outputs_totals.columns = ['true','fitted']

  # Form regression model specification with no intercept
  ols_expr_totals = """true ~ fitted - 1"""
  # fit the OLS regression model
  aux_olsr_results_totals = smf.ols(ols_expr_totals, outputs_totals).fit()
  # store R-squared in rmses list
  ##print('Test R-squared: %.3f' % (aux_olsr_results_totals.rsquared))

  # Compute RMSE
  ac_totals_rmse = DataFrame(ac_totals)
  pc_totals_rmse = DataFrame(pc_totals)
  mse_totals = mean_squared_error(ac_totals_rmse, pc_totals_rmse)
  rmse_totals = sqrt(mse_totals)
  return aux_olsr_results.rsquared,rmse,aux_olsr_results_totals.rsquared,rmse_totals


#@title 5-fold cv on the training data: total outages lags
def cv_fold_w_out_total_lag(all_data_train_vals,out_train_raw):
  # initialize variables
  lookback = 11
  num_splits = 5
  rmses = []
  avgs = []
  k = 0 # starting in CV

  # first iteration : call k_folds() user-defined function
  folds_s = k_folds(all_data_train_vals,num_splits)

  for lags in range(lookback):
    f_test = folds_s[0]
    f_trains = list([folds_s[1][0],folds_s[4][1]])
    fs_tr = f_trains
    fs_tes = f_test
    if lags == 0: # no lags or zero lags
      for fold_splits in range(num_splits):
        # retrieve training and test samples via compute_samples_x_y_zero_train() and compute_samples_x_y_zero_test()
        Xtr , ytr, X_scaler_x, Y_scaler_y = compute_samples_x_y_zero_train(all_data_train_vals,lags,fs_tr)
        Xtes , ytes, y_wind = compute_samples_x_y_zero_test(all_data_train_vals,lags,fs_tes)

        ##print(fs_tes, fs_tr)
        Xtr = DataFrame(Xtr)
        ytr = DataFrame(ytr)
        Xtes = DataFrame(Xtes)
        ytes = DataFrame(ytes)

        if Xtr.isnull().values.any():
          # Interpolate missing values
          Xtr = np.array(Xtr.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))
        if ytr.isnull().values.any():
          # Interpolate missing values
          ytr = np.array(ytr.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))
        if Xtes.isnull().values.any():
          # Interpolate missing values
          Xtes = np.array(Xtes.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))
        if ytes.isnull().values.any():
          # Interpolate missing values
          ytes = np.array(ytes.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))

        X_train = np.array(Xtr)
        y_train = np.array(ytr).ravel()
        X_test = Xtes
        y_test = ytes

        # modeling
        knn = neighbors.KNeighborsRegressor()
        regressor = knn.fit(X_train,y_train)

        # Prediction on validation data
        # Apply transforms to regression variables
        X_test_scale = X_scaler_x.transform(X_test)
        y_test_scale = y_test
        y_test_scale = np.array(y_test_scale)
        y_test_scale = y_test_scale.ravel()
        yhat =  regressor.predict(X_test_scale)
        actual_counts = y_test_scale

        # Inverse response variable transform
        yhat_val = np.array(yhat)
        yhat_reshape = yhat_val.reshape(yhat_val.shape[0],1)
        predicted_counts = Y_scaler_y.inverse_transform(yhat_reshape)
        # Convert to series
        ac = actual_counts.ravel()
        pc = predicted_counts.ravel()


        #OLS regression model with no intercept : just a straight line passing through the origin
        ac_df = DataFrame(ac)
        pc_df = DataFrame(pc)

        true_sum_totals_cv = []
        for n in range(len(y_wind)):
          true_sum_totals_cv.append(np.sum(y_wind[n]))
        tru_totals_cv = pd.concat(true_sum_totals_cv)
        tru_totals_cv = tru_totals_cv.reset_index(drop=True)

        pc_preds = DataFrame(pc)
        forecast = pc_preds
        preds_sum_totals_cv = []
        win_index = 0
        for nk in range(len(y_wind)):
          win_len = len(y_wind[nk])
          preds_sum_totals_cv.append(np.sum(forecast[win_index:win_index + win_len]))
          win_index = win_index + win_len
        preds_totals_cv = pd.concat(preds_sum_totals_cv)
        preds_totals_cv = preds_totals_cv.reset_index(drop=True)

        #OLS regression model with no intercept : just a straight line passing through the origin
        ac_totals = DataFrame(tru_totals_cv)
        pc_totals = DataFrame(preds_totals_cv)
        outputs_totals = pd.concat([ac_totals,pc_totals], axis = 1)
        outputs_totals.columns = ['true','fitted']

        # Form regression model specification with no intercept
        ols_expr_totals = """true ~ fitted - 1"""
        # fit the OLS regression model
        aux_olsr_results_totals = smf.ols(ols_expr_totals, outputs_totals).fit()
        # store R-squared in rmses list
        rmses.append(aux_olsr_results_totals.rsquared)

        # Prepare next CV fold
        if k < 3:
          fs_tes = folds_s[k+1]
          inter_tes_prev = folds_s[k]
          inter_tr = list([folds_s[k+2][0],folds_s[-1][1]])
          fs_tr = list([inter_tes_prev,inter_tr])
        else:
          fs_tes = folds_s[-1]
          fs_tr = list([folds_s[0][0],folds_s[3][1]])
        k = k + 1

      k = 0
      average = Average_cv(rmses)
      avgs.append([lags,average])
      rmses = []
      true_sum_totals_cv = []
      preds_sum_totals_cv = []

    else:  # Where lag is greater than or equals to 1
      for fold_splits in range(num_splits):
        # retrieve training and test samples via compute_samples_x_y_zero_train() and compute_samples_x_y_zero_test()
        Xtr_full , ytr_full, X_scaler_x_full, Y_scaler_y_full = compute_samples_x_y_train_multiple(all_data_train_vals,fs_tr,lags,
                                                                                                   out_train_raw)
        Xtes_full , ytes_full, y_wind = compute_samples_x_y_test_multiple(all_data_train_vals,fs_tes,lags,out_train_raw)

        ##print(fs_tes, fs_tr)
        Xtr_full = DataFrame(Xtr_full)
        ytr_full = DataFrame(ytr_full)
        Xtes_full = DataFrame(Xtes_full)
        ytes_full = DataFrame(ytes_full)

        if Xtr_full.isnull().values.any():
          # Interpolate missing values
          Xtr_full = np.array(Xtr_full.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))
        if ytr_full.isnull().values.any():
          # Interpolate missing values
          ytr_full = np.array(ytr_full.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))

        if Xtes_full.isnull().values.any():
          # Interpolate missing values
          Xtes_full = np.array(Xtes_full.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))
        if ytes_full.isnull().values.any():
          # Interpolate missing values
          ytes_full = np.array(ytes_full.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))

        X_train = np.array(Xtr_full)
        y_train = np.array(ytr_full).ravel()
        X_test = Xtes_full
        y_test = ytes_full

        # modeling
        knn = neighbors.KNeighborsRegressor()
        regressor = knn.fit(X_train,y_train)

        # Prediction on validation data
        # Apply transforms to regression variables
        X_test_scale = X_scaler_x_full.transform(X_test)
        y_test_scale = y_test
        y_test_scale = np.array(y_test_scale)
        y_test_scale = y_test_scale.ravel()
        yhat =  regressor.predict(X_test_scale)
        actual_counts = y_test_scale

        # Inverse response variable transform
        yhat_val = np.array(yhat)
        yhat_reshape = yhat_val.reshape(yhat_val.shape[0],1)
        predicted_counts = Y_scaler_y_full.inverse_transform(yhat_reshape)
        # Convert to series
        ac = actual_counts.ravel()
        pc = predicted_counts.ravel()

        #OLS regression model with no intercept : just a straight line passing through the origin
        ac_df = DataFrame(ac)
        pc_df = DataFrame(pc)

        true_sum_totals_cv = []
        for n in range(len(y_wind)):
          true_sum_totals_cv.append(np.sum(y_wind[n]))
        tru_totals_cv = pd.concat(true_sum_totals_cv)
        tru_totals_cv = tru_totals_cv.reset_index(drop=True)

        pc_preds = DataFrame(pc)
        forecast = pc_preds
        preds_sum_totals_cv = []
        win_index = 0
        for nk in range(len(y_wind)):
          win_len = len(y_wind[nk])
          preds_sum_totals_cv.append(np.sum(forecast[win_index:win_index + win_len]))
          win_index = win_index + win_len
        preds_totals_cv = pd.concat(preds_sum_totals_cv)
        preds_totals_cv = preds_totals_cv.reset_index(drop=True)

        #OLS regression model with no intercept : just a straight line passing through the origin
        ac_totals = DataFrame(tru_totals_cv)
        pc_totals = DataFrame(preds_totals_cv)
        outputs_totals = pd.concat([ac_totals,pc_totals], axis = 1)
        outputs_totals.columns = ['true','fitted']

        # Form regression model specification with no intercept
        ols_expr_totals = """true ~ fitted - 1"""
        # fit the OLS regression model
        aux_olsr_results_totals = smf.ols(ols_expr_totals, outputs_totals).fit()
        # store R-squared in rmses list
        rmses.append(aux_olsr_results_totals.rsquared)

        # Prepare next CV fold
        if k < 3:
          fs_tes = folds_s[k+1]
          inter_tes_prev = folds_s[k]
          inter_tr = list([folds_s[k+2][0],folds_s[-1][1]])
          fs_tr = list([inter_tes_prev,inter_tr])
        else:
          fs_tes = folds_s[-1]
          fs_tr = list([folds_s[0][0],folds_s[3][1]])
        k = k + 1

      k = 0
      average = Average_cv(rmses)
      avgs.append([lags,average])
      rmses = []
      true_sum_totals_cv = []
      preds_sum_totals_cv = []

  idx, max_value= max(avgs, key=lambda item: item[1])

  return idx,max_value


#@title model selection
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors

import statsmodels.api as sm
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from sklearn.metrics import mean_squared_error
from math import sqrt
import patsy
import pickle
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')
from scipy import stats

def cv_fold_w_out_lag(all_data_train_vals,out_train_raw):
  # initialize variables
  lookback = 11
  num_splits = 5
  rmses = []
  avgs = []
  k = 0 # starting in CV

  # first iteration : call k_folds() user-defined function
  folds_s = k_folds(all_data_train_vals,num_splits)

  for lags in range(lookback):
    f_test = folds_s[0]
    f_trains = list([folds_s[1][0],folds_s[4][1]])
    fs_tr = f_trains
    fs_tes = f_test
    if lags == 0: # no lags or zero lags
      for fold_splits in range(num_splits):
        # retrieve training and test samples via compute_samples_x_y_zero_train() and compute_samples_x_y_zero_test()
        Xtr , ytr, X_scaler_x, Y_scaler_y = compute_samples_x_y_zero_train(all_data_train_vals,lags,fs_tr)
        Xtes , ytes, yw = compute_samples_x_y_zero_test(all_data_train_vals,lags,fs_tes)

        ##print(fs_tes, fs_tr)
        Xtr = DataFrame(Xtr)
        ytr = DataFrame(ytr)
        Xtes = DataFrame(Xtes)
        ytes = DataFrame(ytes)

        if Xtr.isnull().values.any():
          # Interpolate missing values
          Xtr = np.array(Xtr.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))
        if ytr.isnull().values.any():
          # Interpolate missing values
          ytr = np.array(ytr.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))
        if Xtes.isnull().values.any():
          # Interpolate missing values
          Xtes = np.array(Xtes.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))
        if ytes.isnull().values.any():
          # Interpolate missing values
          ytes = np.array(ytes.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))

        X_train = np.array(Xtr)
        y_train = np.array(ytr).ravel()
        X_test = Xtes
        y_test = ytes

        # modeling
        knn = neighbors.KNeighborsRegressor()
        regressor = knn.fit(X_train,y_train)

        # Prediction on validation data
        # Apply transforms to regression variables
        X_test_scale = X_scaler_x.transform(X_test)
        ##X_test_scale  = pd.DataFrame(stats.zscore(X_test))
        y_test_scale = y_test
        y_test_scale = np.array(y_test_scale)
        y_test_scale = y_test_scale.ravel()
        yhat =  regressor.predict(X_test_scale)
        actual_counts = y_test_scale

        # Inverse response variable transform
        yhat_val = np.array(yhat)
        yhat_reshape = yhat_val.reshape(yhat_val.shape[0],1)
        predicted_counts = Y_scaler_y.inverse_transform(yhat_reshape)
        # Convert to series
        ac = actual_counts.ravel()
        pc = predicted_counts.ravel()

        #OLS regression model with no intercept : just a straight line passing through the origin
        ac_df = DataFrame(ac)
        pc_df = DataFrame(pc)
        outputs = pd.concat([ac_df,pc_df], axis = 1)
        outputs.columns = ['true','fitted']

        # Form regression model specification with no intercept
        ols_expr = """true ~ fitted - 1"""
        # fit the OLS regression model
        aux_olsr_results = smf.ols(ols_expr, outputs).fit()
        # store R-squared in rmses list
        rmses.append(aux_olsr_results.rsquared)

        # Prepare next CV fold
        if k < 3:
          fs_tes = folds_s[k+1]
          inter_tes_prev = folds_s[k]
          inter_tr = list([folds_s[k+2][0],folds_s[-1][1]])
          fs_tr = list([inter_tes_prev,inter_tr])
        else:
          fs_tes = folds_s[-1]
          fs_tr = list([folds_s[0][0],folds_s[3][1]])
        k = k + 1

      k = 0

      average = Average_cv(rmses)
      avgs.append([lags,average])
      rmses = []

    else:  # Where lag is greater than or equals to 1
      for fold_splits in range(num_splits):
        # retrieve training and test samples via compute_samples_x_y_zero_train() and compute_samples_x_y_zero_test()
        Xtr_full , ytr_full, X_scaler_x_full,Y_scaler_y_full = compute_samples_x_y_train_multiple(all_data_train_vals,fs_tr,lags,
                                                                                                  out_train_raw)
        Xtes_full , ytes_full, yw_full = compute_samples_x_y_test_multiple(all_data_train_vals,fs_tes,lags,out_train_raw)

        ##print(fs_tes, fs_tr)
        Xtr_full = DataFrame(Xtr_full)
        ytr_full = DataFrame(ytr_full)
        Xtes_full = DataFrame(Xtes_full)
        ytes_full = DataFrame(ytes_full)

        if Xtr_full.isnull().values.any():
          # Interpolate missing values
          Xtr_full = np.array(Xtr_full.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))
        if ytr_full.isnull().values.any():
          # Interpolate missing values
          ytr_full = np.array(ytr_full.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))

        if Xtes_full.isnull().values.any():
          # Interpolate missing values
          Xtes_full = np.array(Xtes_full.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))
        if ytes_full.isnull().values.any():
          # Interpolate missing values
          ytes_full = np.array(ytes_full.interpolate(method ='linear', limit_direction ='forward', limit = 100, axis=0))

        X_train = np.array(Xtr_full)
        y_train = np.array(ytr_full).ravel()
        X_test = Xtes_full
        y_test = ytes_full

        # modeling
        knn = neighbors.KNeighborsRegressor()
        regressor = knn.fit(X_train,y_train)

        # Prediction on validation data
        # Apply transforms to regression variables
        X_test_scale = X_scaler_x_full.transform(X_test)
        y_test_scale = y_test
        y_test_scale = np.array(y_test_scale)
        y_test_scale = y_test_scale.ravel()
        yhat =  regressor.predict(X_test_scale)
        actual_counts = y_test_scale

        # Inverse response variable transform
        yhat_val = np.array(yhat)
        yhat_reshape = yhat_val.reshape(yhat_val.shape[0],1)
        predicted_counts = Y_scaler_y_full.inverse_transform(yhat_reshape)

        # Convert to series
        ac = actual_counts.ravel()
        pc = predicted_counts.ravel()

        #OLS regression model with no intercept : just a straight line passing through the origin
        ac_df = DataFrame(ac)
        pc_df = DataFrame(pc)
        outputs = pd.concat([ac_df,pc_df], axis = 1)
        outputs.columns = ['true','fitted']

        # Form regression model specification with no intercept
        ols_expr = """true ~ fitted - 1"""
        # fit the OLS regression model
        aux_olsr_results = smf.ols(ols_expr, outputs).fit()
        # store R-squared in rmses list
        rmses.append(aux_olsr_results.rsquared)

        # Prepare next CV fold
        if k < 3:
          fs_tes = folds_s[k+1]
          inter_tes_prev = folds_s[k]
          inter_tr = list([folds_s[k+2][0],folds_s[-1][1]])
          fs_tr = list([inter_tes_prev,inter_tr])
        else:
          fs_tes = folds_s[-1]
          fs_tr = list([folds_s[0][0],folds_s[3][1]])
        k = k + 1

      k = 0
      average = Average_cv(rmses)
      ##avgs.append((average))
      avgs.append([lags,average])
      rmses = []
  idx, max_value= max(avgs, key=lambda item: item[1])
  return idx, max_value

#@title get_lags_train
def get_lags(extend_samples_tr,la_tr,raw_train):
  lags_full_data = []
  ty_lag_tr = extend_samples_tr.iloc[[0]]['id']
  ty_lag_tr = np.array(ty_lag_tr)
  ty_lag_tr = ty_lag_tr.item()

  ty_lag_las = extend_samples_tr.iloc[[-1]]['id']
  ty_lag_tr_las = np.array(ty_lag_las)
  ty_lag_tr_las = ty_lag_tr_las.item()

  start_lag_tr = ty_lag_tr
  end_lag_tr = abs(start_lag_tr - la_tr)
  lags_df  = raw_train.values

  for val_lags in range(end_lag_tr,ty_lag_tr_las+1):
    values = lags_df[val_lags]
    values_df = DataFrame(values)
    lags_full_data.append(values_df.T)

  all_data_slags = []
  for z in range(len(lags_full_data)):
    ds_lags = pd.DataFrame(lags_full_data[z])
    slags_df = pd.concat([ds_lags])
    slags_df.columns = ['Temp','Preci','Gust','Wind','Windir','LAI','Total Outages']
    all_data_slags.append(slags_df)
  slagFull_set = pd.concat(all_data_slags)
  return pd.concat([slagFull_set]).reset_index(drop=True)

#@title regression samples _ vals_lags
import patsy
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf
from math import sqrt

def mod_vals_lags(model,X_scaler_tr,Y_scaler_tr,data_c,lg,rwtest):
  X_tests = []
  y_tests = []

  for full_tes in range(len(data_c)):
    df_ws_test_data = data_c[full_tes][['id','Temp','Preci','Gust','Wind','Windir','LAI']]
    df_outage_test_data_d = data_c[full_tes][['Total Outages']]

    df_lags_test_data = get_lags_test(df_ws_test_data,lg,rwtest)
    df_lags_test_data_idw = df_lags_test_data.drop('id', axis=1)
    df_lags_test_data_tot = df_lags_test_data_idw.drop('Total Outages', axis=1)
    df_last_drop = df_lags_test_data_tot

    if df_last_drop.empty:
      None
    else:
      # add relative normalized time points
      p_f_array = np.arange(len(df_last_drop))
      df_last_drop['norm_val'] = [(x+1)/len(df_last_drop) for x in p_f_array]
      df_last_drop = df_last_drop[['Temp','Preci','Gust','Wind','Windir','LAI','norm_val']]

      df_last_drop = df_last_drop.reset_index(drop=True)
      df_last_drop_vals = df_last_drop.values

      df_outage_test_data_d = df_outage_test_data_d.reset_index(drop=True)
      df_outage_test_data_d_vals = df_outage_test_data_d.values

      xtest , ytest = split_sequences_cv(df_last_drop_vals,df_outage_test_data_d_vals,lg+1)

      if not xtest:
        None
      else:
        X_tests.append(xtest)
        y_tests.append(pd.DataFrame(ytest))

  sample_X_test = np.vstack(X_tests)
  sample_y_test = pd.concat(y_tests)
  sample_y_test = np.array(sample_y_test)
  sample_X_test_reshape = sample_X_test.reshape(sample_X_test.shape[0],sample_X_test.shape[1]*sample_X_test.shape[-1])

  #@title hourly outages: 2
  # Apply transforms on test data: hourly outages
  xtr_scale_test = X_scaler_tr.transform(sample_X_test_reshape)

  d_x_test = DataFrame(xtr_scale_test)
  d_y_test = DataFrame(sample_y_test).reset_index(drop=True)
  data_fuse_test = pd.concat([d_x_test,d_y_test],axis=1)
  f_test = 'd_y_test~d_x_test - 1'
  yst , Xst = patsy.dmatrices(f_test, data_fuse_test, return_type='matrix')
  X_test_apply = Xst
  y_test_apply = yst.astype(int)

  # make prediction on the transformed test data
  yhat = model.predict(X_test_apply)
  actual_counts = y_test_apply
  # Inverse response variable transform
  yhat_val = np.array(yhat)
  yhat_reshape = yhat_val.reshape(yhat_val.shape[0],1)
  predicted_counts = Y_scaler_tr.inverse_transform(yhat_reshape)

  # Convert to series
  actual_counts = np.array(actual_counts)
  ac = actual_counts.ravel()
  predicted_counts = np.array(predicted_counts)
  pc = predicted_counts.ravel()

  #OLS regression model with no intercept : just a straight line passing through the origin
  ac_df = DataFrame(ac)
  pc_df = DataFrame(pc)
  outputs = pd.concat([ac_df,pc_df], axis = 1)
  outputs.columns = ['true','fitted']
  ##outputs.to_csv('file.csv')

  # Form regression model specification with no intercept
  ols_expr = """true ~ fitted - 1"""
  # fit the OLS regression model
  aux_olsr_results = smf.ols(ols_expr, outputs).fit()

  # Compute RMSE
  ac_df_rmse = DataFrame(ac)
  pc_df_rmse = DataFrame(pc)
  mse = mean_squared_error(ac_df_rmse, pc_df_rmse)
  rmse = sqrt(mse)

  #####################################
  # Total Outages
  #####################################
  wind_observ = []

  true_sum_totals = []
  for n in range(len(y_tests)):
    true_sum_totals.append(np.sum(y_tests[n]))
  tru_totals = pd.concat(true_sum_totals)
  tru_totals = tru_totals.reset_index(drop=True)

  pc_preds = DataFrame(pc)
  forecast = pc_preds
  preds_sum_totals = []
  win_index = 0
  for nk in range(len(y_tests)):
    win_len = len(y_tests[nk])
    # sum forecast windows
    preds_sum_totals.append(np.sum(forecast[win_index:win_index + win_len]))
    wind_observ.append(forecast[win_index:win_index + win_len])
    win_index = win_index + win_len
  preds_totals = pd.concat(preds_sum_totals)
  preds_totals = preds_totals.reset_index(drop=True)

  #OLS regression model with no intercept : just a straight line passing through the origin
  ac_totals = DataFrame(tru_totals)
  pc_totals = DataFrame(preds_totals)
  outputs_totals = pd.concat([ac_totals,pc_totals], axis = 1)
  outputs_totals.columns = ['true','fitted']
  # Save output file
  ##outputs_totals.to_csv('file.csv')

  # Form regression model specification with no intercept
  ols_expr_totals = """true ~ fitted - 1"""
  # fit the OLS regression model
  aux_olsr_results_totals = smf.ols(ols_expr_totals, outputs_totals).fit()
  # store R-squared in rmses list
  ##print('Test R-squared: %.3f' % (aux_olsr_results_totals.rsquared))

  # Compute RMSE
  ac_totals_rmse = DataFrame(ac_totals)
  pc_totals_rmse = DataFrame(pc_totals)
  mse_totals = mean_squared_error(ac_totals_rmse, pc_totals_rmse)
  rmse_totals = sqrt(mse_totals)

  return aux_olsr_results.rsquared,rmse,aux_olsr_results_totals.rsquared,rmse_totals


#@title get_lags_train
def get_lags(extend_samples_tr,la_tr,raw_train):
  lags_full_data = []
  ty_lag_tr = extend_samples_tr.iloc[[0]]['id']
  ty_lag_tr = np.array(ty_lag_tr)
  ty_lag_tr = ty_lag_tr.item()

  ty_lag_las = extend_samples_tr.iloc[[-1]]['id']
  ty_lag_tr_las = np.array(ty_lag_las)
  ty_lag_tr_las = ty_lag_tr_las.item()

  start_lag_tr = ty_lag_tr
  end_lag_tr = abs(start_lag_tr - la_tr)
  lags_df  = raw_train.values

  for val_lags in range(end_lag_tr,ty_lag_tr_las+1):
    values = lags_df[val_lags]
    values_df = DataFrame(values)
    lags_full_data.append(values_df.T)

  all_data_slags = []
  for z in range(len(lags_full_data)):
    ds_lags = pd.DataFrame(lags_full_data[z])
    slags_df = pd.concat([ds_lags])
    slags_df.columns = ['Temp','Preci','Gust','Wind','Windir','LAI','Total Outages']
    all_data_slags.append(slags_df)
  slagFull_set = pd.concat(all_data_slags)
  return pd.concat([slagFull_set]).reset_index(drop=True)

#@title get lags test data
def get_lags_test(d_ws,la,df_lag):
  ty_lag = d_ws.iloc[[0]]['id']
  ty_lag = np.array(ty_lag)
  ty_lag = ty_lag.item()

  ty_lag_e = d_ws.iloc[[-1]]['id']
  ty_lag_e = np.array(ty_lag_e)
  ty_lag_e = ty_lag_e.item()

  start_lag = ty_lag
  end_lag = abs(start_lag - la)

  orig_data_lag = df_lag.loc[(df_lag['id']>=end_lag) & (df_lag['id']<=ty_lag_e)]
  orig_data_lag.columns = ['id','Temp','Preci','Gust','Wind','Windir','LAI','Total Outages']
  return orig_data_lag

#@title compute weather storm intervals -  gap concepts
def wea_outlier_r(outlier_data,g):
  # iForest based on matched trained data
  outlier_data['id'] = outlier_data.index
  outlier_data.columns = ['Temp_min','Temp_max','Preci_max','Preci_avg','Gust_max','Gust_avg',
                            'Wind_max','Wind_avg','Windir_avg','Windir_std','LAI_avg','scores','anomaly','id']
  # rearrange columns
  outlier_data = outlier_data[['id','Temp_min','Temp_max','Preci_max','Preci_avg','Gust_max','Gust_avg',
                          'Wind_max','Wind_avg','Windir_avg','Windir_std','LAI_avg','scores','anomaly']]

  tes_outlier = outlier_data
  # storm windows based on iForest outlier labels:
  ids_labs_gabs = []

  count = 0
  count_r = 0
  count_l = 0
  n_r = 0
  n_l = 0
  m = 0
  count_t  = 0

  for k in range(len(tes_outlier)):
    if m >= len(tes_outlier):
          break
    while not tes_outlier['anomaly'].iloc[m].item() == 0:
      ids_labs_gabs.append(tes_outlier['anomaly'].iloc[m].item())
      m = m + 1
      if m >= len(tes_outlier):
          break

    if m >= len(tes_outlier):
          break

    if tes_outlier['anomaly'].iloc[m].item()== 0:
      n_r = m
      n_l = m
      count_r = count_r + 1
      count_l = count_l + 1

    while tes_outlier['anomaly'].iloc[n_r].item()== 0:
      count_r = count_r + 1
      n_r = n_r + 1

      if n_r >= len(tes_outlier):
        break

    while tes_outlier['anomaly'].iloc[n_l].item()== 0:
      count_l = count_l + 1
      n_l = n_l - 1

    count = count_r + count_l
    if count <= g:
      ids_labs_gabs.append(1)
    else:
      ids_labs_gabs.append(0)

    m = m + 1
    count = 0
    count_r = 0
    count_l = 0

  labs_storm_df = DataFrame(ids_labs_gabs)
  arr_labs = np.array(labs_storm_df)
  arr_labs = arr_labs.ravel()
  outlier_data['anomaly'] = arr_labs
  outlier_data.drop(columns=['scores'], inplace=True)
  return  outlier_data

# main function
#@title gridsearch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from patsy import dmatrices
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pandas import DataFrame
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.model_selection import TimeSeriesSplit
import seaborn as sns

from sklearn.metrics import make_scorer, f1_score
from sklearn import model_selection

sns.set_style('white')
%matplotlib inline

##!pip install -U kaleido

win = 6
overlap = 5
rng = np.random.RandomState(42)

mydata = pd.read_csv('nas.csv',header=0)
dataIn = mydata[mydata.columns[1:8]]
dateT = mydata[mydata.columns[0]]
dateT = DataFrame(dateT)
dateT['dateTime'] = dateT
dateT.drop('Unnamed: 0', axis = 1, inplace = True)
df_data = pd.concat((dateT, dataIn), axis=1)
df_data = df_data.drop(df_data.columns[0], axis = 1)

# Drop rows which contain any NaN values in all columns
df_data = df_data.dropna( how='all')

# Interpolate missing values
df_data = df_data.interpolate(method ='linear', limit_direction ='both', limit = 10000, axis=0)

# move target variable to the last column to align with surpervise learning struture
finData = pd.DataFrame(df_data,columns=['Temp','Preci','Gust','Wind','Windir','LAI','Total Outages'])

# Clean dataset
finData[finData['Preci']<0]  = 0
finData['LAI'][10565] = 0.7
# replace with NANs
finData['Temp'][1800:2700] = np.nan
finData['Preci'][1800:2700] = np.nan
finData['Gust'][1800:2700] = np.nan
finData['Wind'][1800:2700] = np.nan
finData['Windir'][1800:2700] = np.nan

# Interpolate missing values
w_out_data = finData.interpolate(method ='linear', limit_direction ='forward', limit = 10000, axis=0)
nas_wea_per = w_out_data

# split into train and test set
outlier_train = nas_wea_per
out_size = int(len(outlier_train) * .8)
df_outlier_tr, df_outlier_tes = outlier_train[:out_size], outlier_train[out_size:]
raw_tr_val = df_outlier_tr
df_outlier_tr_drop = df_outlier_tr.drop('Total Outages', axis = 1)
df_outlier_tes_drop = df_outlier_tes.drop('Total Outages', axis = 1)

# Read probabilistic - high outage interval data
d_high = pd.read_csv('file',header=0,index_col=0)

# Create validation data for conbination of hyperparamenters
df_train = df_outlier_tr_drop[:int(df_outlier_tr_drop.shape[0]*0.8)]
raw_s_train = df_outlier_tr[:int(df_outlier_tr.shape[0]*0.8)]
df_val = df_outlier_tr_drop[len(df_train):]
raw_val = df_outlier_tr[len(df_train):]

# high outage intervals
d_high_out_tr = d_high[:int(d_high.shape[0]*0.8)]
d_high_out_val = d_high[len(d_high_out_tr):]

raw_s_train['id'] = raw_s_train.index
raw_s_train.columns = [['Temp','Preci','Gust','Wind','Windir','LAI','Total Outages','id']]
# rearrange columns
raw_s_train = raw_s_train[['id','Temp','Preci','Gust','Wind','Windir','LAI','Total Outages']]

raw_val['id'] = raw_val.index
raw_val.columns = [['Temp','Preci','Gust','Wind','Windir','LAI','Total Outages','id']]
# rearrange columns
raw_val = raw_val[['id','Temp','Preci','Gust','Wind','Windir','LAI','Total Outages']]

# Break training data into windows: 6hrs and 5hrs overlap
wea_train_data = mapping_series(df_train,win,overlap)
# Break validation data into windows: 6hrs and 5hrs overlap
wea_val_data = mapping_series(df_val,win,overlap)

# fit iForest model to validation data
val_iforest = ifor_vals(wea_val_data)
val_iforest.drop(columns=['scores'], inplace=True)

# matched outages to weather
mat_val_w_out_data,out_data_vals = outs_weas(val_iforest,d_high_out_val,raw_val)

# grid list
conta = [0.1, 0.2, 0.3, 0.4, 0.5]
gaps = [1, 5, 8, 20, 40]
rmses = []
r2s = []
gaps_lst = []
cons = []

# fit the model
for u in range(len(conta)):
    for z in range(len(gaps)):
        clf =IsolationForest(max_samples='auto',contamination=float(conta[u]),random_state=rng)
        clf.fit(wea_train_data)
        wea_pd = pd.DataFrame(wea_train_data)
        new_wea_train_data = wea_pd.copy()
        new_wea_train_data['scores'] = clf.decision_function(wea_pd)
        new_wea_train_data['anomaly'] = clf.predict(wea_pd)
        new_wea_train_data['anomaly'] = new_wea_train_data['anomaly'].map({1:0,-1:1})

        # compute contiguous weather outlier regions
        gapped_data = wea_outlier_r(new_wea_train_data,gaps[z])
        mat_tr_w_out_data,out_data_trains = outs_weas(gapped_data,d_high_out_tr,raw_s_train)

        # compute best set of lag parameters: CV
        drop_id_raw_train = out_data_trains.drop('id',axis=1)
        # total outages
        lgt,max_t = cv_fold_w_out_total_lag(mat_tr_w_out_data,drop_id_raw_train)

        if lgt == 0:
          model, x_scal, y_scal = modeling_tr(mat_tr_w_out_data,lgt)
          r_sq,rme,r_sq_tot,rme_tot = mod_vals(model,x_scal,y_scal,mat_val_w_out_data,lgt)
          print('Contamination:', conta[u], '; Gaps:', gaps[z], '; R_squared:',round(r_sq,3),'; RMSE:',round(rme,3))
          r2s.append([r_sq,conta[u],gaps[z],lgt,rme,max_t])

        else:
          id_raw_train = out_data_trains
          ##model, x_scal, y_scal = modeling_tr(mat_tr_w_out_data,lgt)
          model, x_scal, y_scal = modeling_tr_lags(mat_tr_w_out_data,lgt,id_raw_train)
          id_raw_test = out_data_vals
          r_sq,rme,r_sq_tot,rme_tot = mod_vals_lags(model,x_scal,y_scal,mat_val_w_out_data,lgt,id_raw_test)
          print('Contamination:', conta[u], '; Gaps', gaps[z], '; R_squared:',round(r_sq,3),'; RMSE:',round(rme,3))
          r2s.append([r_sq,conta[u],gaps[z],lgt,rme,max_t])

r2_value,ct,gz,la,rme_value,cv_val = max(r2s, key=lambda item: item[0])
print('Best:','R_squared:', round(r2_value,3), "; Contamination:",ct, "; Gaps:",gz, "; Lag:",la, '; RMSE:', round(rme_value,3) )
# save hyperameters
with open('file','wb') as ftes:
          pickle.dump(r2s,ftes)

Contamination: 0.1 ; Gaps: 1 ; R_squared: 0.071 ; RMSE: 329.909
Contamination: 0.1 ; Gaps 5 ; R_squared: 0.064 ; RMSE: 313.796
Contamination: 0.1 ; Gaps 8 ; R_squared: 0.087 ; RMSE: 318.566
Contamination: 0.1 ; Gaps 20 ; R_squared: 0.026 ; RMSE: 336.87
Contamination: 0.1 ; Gaps 40 ; R_squared: 0.026 ; RMSE: 360.975
Contamination: 0.2 ; Gaps 1 ; R_squared: 0.027 ; RMSE: 355.998
Contamination: 0.2 ; Gaps 5 ; R_squared: 0.033 ; RMSE: 348.649
Contamination: 0.2 ; Gaps: 8 ; R_squared: 0.096 ; RMSE: 307.038
Contamination: 0.2 ; Gaps: 20 ; R_squared: 0.082 ; RMSE: 321.753
Contamination: 0.2 ; Gaps 40 ; R_squared: 0.028 ; RMSE: 335.692
Contamination: 0.3 ; Gaps 1 ; R_squared: 0.043 ; RMSE: 300.248
Contamination: 0.3 ; Gaps 5 ; R_squared: 0.058 ; RMSE: 306.577
Contamination: 0.3 ; Gaps 8 ; R_squared: 0.029 ; RMSE: 401.063
Contamination: 0.3 ; Gaps: 20 ; R_squared: 0.082 ; RMSE: 315.491
Contamination: 0.3 ; Gaps: 40 ; R_squared: 0.077 ; RMSE: 316.819
Contamination: 0.4 ; Gaps 1 ; R_squared: 0.03